# Script de procesamiento de datos

a partir de los datos obtenidos desde Whisper via audios de youtube, limpiaremos los datos y los preparemos para la base de datos sqlite.

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("data.csv")

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
def limpiar_puntos(texto):
    if pd.isna(texto): return texto
    partes = str(texto).split('.')
    if len(partes) > 2:
        # Une la primera parte con la segunda mediante un punto, ignora el resto
        return f"{partes[0]}.{partes[1]}"
    return texto

data['start'] = data['start'].apply(limpiar_puntos).astype(float)
data['end'] = data['end'].apply(limpiar_puntos).astype(float)

In [ ]:
# limpieza de textos vacíos
data['text'] = data['text'].str.strip()
data = data.replace('', np.nan).dropna()
data.info()

In [ ]:

# Limpiamos el formato extraño de puntos antes de empezar
data['start'] = pd.to_numeric(data['start'].astype(str).str.replace(r'\.(?=\d{2}\.)', '', regex=True), errors='coerce')
data['end']   = pd.to_numeric(data['end'].astype(str).str.replace(r'\.(?=\d{2}\.)', '', regex=True), errors='coerce')


In [ ]:
# intervalo de merge de 2 segundos: start < 2s

def segmentacion_estricta(df):
    bloques = []
    texto_acumulado = []
    tiempo_inicio = None
    
    signos_cierre = ('.', '!', '?', '…')

    for i, row in df.iterrows():
        # Guardamos el inicio del primer segmento del bloque
        if tiempo_inicio is None:
            tiempo_inicio = row['start']
        
        texto_actual = str(row['text']).strip()
        texto_acumulado.append(texto_actual)
        
        # SI el texto termina en puntuación, cerramos el bloque
        if texto_actual.endswith(signos_cierre):
            bloques.append({
                'id_video': row['id_video'],
                'start': tiempo_inicio,
                'end': row['end'],
                'text': " ".join(texto_acumulado)
            })
            # Reiniciamos para el siguiente bloque
            texto_acumulado = []
            tiempo_inicio = None
            
    # Manejo de texto huérfano (si el video termina sin punto final)
    if texto_acumulado:
        bloques.append({
            'id_video': df.iloc[-1]['id_video'],
            'start': tiempo_inicio,
            'end': df.iloc[-1]['end'],
            'text': " ".join(texto_acumulado)
        })
        
    return pd.DataFrame(bloques)



In [ ]:
# Uso
df_final = segmentacion_estricta(data)
def generar_url(row):
    # Convertimos a entero porque YouTube no procesa milisegundos en la URL
    segundos = int(row['start'])
    return f"https://www.youtube.com/watch?v={row['id_video']}&t={segundos}s"

# Creamos la nueva columna
df_final['link'] = df_final.apply(generar_url, axis=1)


In [ ]:
df_final

In [ ]:
tokensearch = r'(?=.*abuelo)'
res = df_final[df_final['text'].str.contains(tokensearch, regex=True, case=False, na=False)]
res

# Risómetro [test]

> es muy pesado ejecutar esto. probar un un subset muy pequeño df_final.iloc[:20]

In [ ]:
#!pip install pydub opencv-python

In [ ]:
def medir_risa_local(id_video, start, end):
    # 1. Construir la ruta al archivo que ya tienes
    ruta_audio = f"audios/video_{id_video}.mp3"
    
    # Verificamos si el archivo existe 
    if not os.path.exists(ruta_audio):
        return None 
    
    # 2. Cargar el audio completo
    audio_completo = AudioSegment.from_mp3(ruta_audio)
    
    # 3. Definir el segmento de la risa (3 segundos después del 'end')
    punto_corte_ms = end * 1000
    fin_risa_ms = (end + 3.36) * 1000
    
    # Extraer el segmento de la reacción
    audio_reaccion = audio_completo[punto_corte_ms : fin_risa_ms]
    
    # 4. Calcular el volumen (dBFS)
    # Si el segmento es puro silencio o muy corto, dBFS devuelve -inf
    volumen = audio_reaccion.dBFS
    
    return volumen

#df_2 = df_final.iloc[:20]
#df_2['puntuacion_risa'] = df_2.apply(lambda r: medir_risa_local(r['id_video'], r['start'], r['end']), axis=1)

In [ ]:
# Ver los 5 chistes más exitosos
mejores_chistes = df_2.sort_values('puntuacion_risa', ascending=False).head(5)

for i, row in mejores_chistes.iterrows():
    print(f"Risa ({row['puntuacion_risa']:.2f} dB): {row['text']}")